<a href="https://colab.research.google.com/github/rickiepark/the-lm-book/blob/main/emotion_classifier_LR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="display: flex; justify-content: center;">
    <div style="background-color: #f4f6f7; padding: 15px; width: 80%;">
        <table style="width: 100%">
            <tr>
                <td style="vertical-align: middle;">
                    <span style="font-size: 14px;">
                        A notebook for <a href="https://www.thelmbook.com" target="_blank" rel="noopener">The Hundred-Page Language Models Book</a> by Andriy Burkov<br><br>
                        Code repository: <a href="https://github.com/rickiepark/the-lm-book" target="_blank" rel="noopener">https://github.com/rickiepark/the-lm-book</a>
                    </span>
                </td>
                <td style="vertical-align: middle;">
                    <a href="https://www.thelmbook.com" target="_blank" rel="noopener">
                        <img src="https://thelmbook.com/img/book.png" width="80px" alt="The Hundred-Page Language Models Book">
                    </a>
                </td>
            </tr>
        </table>
    </div>
</div>

In [1]:
# 필수 라이브러리 임포트
import gzip             # gzip 압축 데이터 파일 해제를 위한 라이브러리
import json             # JSON 형식 데이터 구문 분석을 위한 라이브러리
import random           # 데이터셋 셔플링 및 시드 설정을 위한 라이브러리
import requests         # URL에서 데이터셋 다운로드를 위한 라이브러리
from sklearn.feature_extraction.text import CountVectorizer # 텍스트 벡터화 유틸리티
from sklearn.linear_model import LogisticRegression         # 로지스틱 회귀 모델
from sklearn.metrics import accuracy_score                  # 모델 평가를 위한 라이브러리

# ----------------------------
# 유틸리티 함수
# ----------------------------
def set_seed(seed):
    """
    재현성을 위해 난수 시드를 설정합니다.
    Args:
        seed (int): 난수 생성을 위한 시드 값
    """
    random.seed(seed)

def download_and_split_data(data_url, test_ratio=0.1):
    """
    URL에서 감정 분류 데이터셋을 다운로드하고 훈련/테스트 세트로 분할합니다.
    원시 데이터의 압축 해제 및 JSON 구문 분석을 처리합니다.
    Args:
        data_url (str): gzip 압축된 JSON 데이터셋의 URL
        test_ratio (float): 테스트에 사용할 데이터 비율 (기본값: 0.1)
    Returns:
        tuple: (X_train, y_train, X_test, y_test) 포함:
            - X_train, X_test: 훈련 및 테스트를 위한 텍스트 샘플 목록
            - y_train, y_test: 해당 감정 레이블 목록
    """
    # 데이터셋 다운로드 및 압축 해제
    response = requests.get(data_url)
    content = gzip.decompress(response.content).decode()
    # JSON 라인을 딕셔너리의 리스트로 파싱합니다.
    dataset = [json.loads(line) for line in content.splitlines()]
    # 무작위 분할을 위해 데이터셋 셔플링
    random.shuffle(dataset)
    # 훈련 및 테스트 세트로 분할
    split_index = int(len(dataset) * (1 - test_ratio))
    train, test = dataset[:split_index], dataset[split_index:]
    # 텍스트와 레이블 분리
    X_train = [item["text"] for item in train]
    y_train = [item["label"] for item in train]
    X_test = [item["text"] for item in test]
    y_test = [item["label"] for item in test]
    return X_train, y_train, X_test, y_test

# ----------------------------
# 메인 실행
# ----------------------------
# 재현성을 위해 난수 시드 설정
set_seed(42)

# 데이터셋 다운로드 및 준비
data_url = "https://www.thelmbook.com/data/emotions"
X_train_text, y_train, X_test_text, y_test = download_and_split_data(
    data_url, test_ratio=0.1
)
print("훈련 샘플 수:", len(X_train_text))
print("테스트 샘플 수:", len(X_test_text))

# ----------------------------
# 기준 모델
# ----------------------------
# 기본 매개변수로 텍스트 벡터라이저 초기화
# max_features=10_000: 어휘사전을 상위 10k개의 가장 빈번한 단어로 제한
# binary=True: 개수를 이진 표시기(0/1)로 변환
vectorizer = CountVectorizer(max_features=10_000, binary=True)

# 텍스트 데이터를 수치 특성으로 변환
X_train = vectorizer.fit_transform(X_train_text)
X_test = vectorizer.transform(X_test_text)

# 로지스틱 회귀 모델 초기화 및 훈련
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train, y_train)

# 훈련 및 테스트 세트에서 예측 수행
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# 정확도 메트릭 계산 및 표시
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"\n훈련 정확도: {train_accuracy:.4f}")
print(f"테스트 정확도: {test_accuracy:.4f}")

# ----------------------------
# 개선된 모델
# ----------------------------
print("\n--- 더 나은 하이퍼파라미터 ---")
# 개선된 매개변수로 벡터라이저 초기화
# max_features=20000: 증가된 어휘 크기
# ngram_range=(1, 2): 유니그램과 바이그램 모두 포함
vectorizer = CountVectorizer(max_features=20000, ngram_range=(1, 2))

# 새 벡터라이저로 텍스트 데이터 변환
X_train = vectorizer.fit_transform(X_train_text)
X_test = vectorizer.transform(X_test_text)

# 동일한 매개변수로 모델 훈련 및 평가
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"훈련 정확도: {train_accuracy:.4f}")
print(f"테스트 정확도: {test_accuracy:.4f}")

훈련 샘플 수: 18000
테스트 샘플 수: 2000

훈련 정확도: 0.9854
테스트 정확도: 0.8880

--- 더 나은 하이퍼파라미터 ---
훈련 정확도: 0.9963
테스트 정확도: 0.8890
